In [1]:
import numpy as np 
import pandas as pd
import csv
import os
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import random
import emoji
import string
import json
import nltk

from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

# list of all the dataset files

dataset_paths = ["/INET/state-trolls/work/state-trolls/datasets/russia_052020_tweets_csv_hashed_2.csv", 
         "/INET/state-trolls/work/state-trolls/datasets/russian_linked_tweets_csv_hashed.csv", 
         "/INET/state-trolls/work/state-trolls/datasets/ira_tweets_csv_hashed.csv", 
         "/INET/state-trolls/work/state-trolls/datasets/russia_201906_1_tweets_csv_hashed.csv"]

# path to store the entire combined dataset
combined_dataset_path = "../datasets/russian_trolls.csv"

# returns a pandas dataframe consisting of entries from all the dataset files
def get_combined_dataset(paths):
    data = pd.concat((pd.read_csv(file) for file in tqdm(paths)))
    return data

data = get_combined_dataset(dataset_paths)
print("Number of tweets in the dataset: ", data.shape[0])

# extracts just the english tweets by using the language tag
is_english_tweet = data['tweet_language'] == 'en'
english_data = data[is_english_tweet]

print("Number of English tweets in the dataset: ", english_data.shape[0])
english_tweet_data = english_data[['tweetid', 'tweet_text']]

# takes list of tweets as input and returns list of pre-processed tweets as output
def preprocess(tweets):
    processed_tweets = []
    for tweet in tweets:
        result = re.sub(r"http\S+", "", tweet)
        result = re.sub(r"RT @\S+", "", result)
        result = re.sub(r"@\S+", "", result)
        result = re.sub(emoji.get_emoji_regexp(), "", result)
        result_removed_punctuation = result.translate(str.maketrans('', '', string.punctuation))
        result = re.sub(r"[^a-zA-Z0-9 ]", "", result_removed_punctuation)
        processed_tweets.append(result)
    return processed_tweets

# Filtering data between 01.01.2019 to 01.07.2017
data['tweet_time'] = pd.to_datetime(data['tweet_time'], format = '%Y-%m-%d')
start_date = '2016-01-01'
end_date = '2017-07-01'
mask = (data['tweet_time'] > start_date) & (data['tweet_time'] <= end_date)
data = data.loc[mask]
print("Number of tweets in the dataset after filtering: ", data.shape[0])


# extracts just the english tweets by using the language tag
is_english_tweet = data['tweet_language'] == 'en'
f_english_data = data[is_english_tweet]
print("Number of English tweets in the filtered dataset: ", f_english_data.shape[0])
f_english_tweet_data = f_english_data[['tweetid', 'tweet_text']]

tweets = f_english_tweet_data['tweet_text']
tweets = preprocess(tweets)

f_english_tweet_data = f_english_tweet_data.assign(processed_tweets = tweets)

# removes the entries having empty string after preprocessing
is_not_empty_string = f_english_tweet_data['processed_tweets'].apply(lambda x: not str.isspace(x))
f_english_tweet_data = f_english_tweet_data[is_not_empty_string]
print("Number of english tweets after filtering and preprocessing before dropping the duplicates: ", f_english_tweet_data.shape[0])
f_english_tweet_data.drop_duplicates(subset ="tweetid", keep = 'first', inplace = True) 
f_english_tweet_data = f_english_tweet_data.reset_index()
print("Number of english tweets after filtering and preprocessing and dropping the duplicates: ", f_english_tweet_data.shape[0])
twitter_data = f_english_tweet_data


100%|██████████| 4/4 [00:54<00:00, 13.60s/it]


Number of tweets in the dataset:  9995700
Number of English tweets in the dataset:  3739633
Number of tweets in the dataset after filtering:  3170632
Number of English tweets in the filtered dataset:  1759076
Number of english tweets after filtering and preprocessing before dropping the duplicates:  1757447
Number of english tweets after filtering and preprocessing and dropping the duplicates:  1662395


In [2]:
# ## Set up of PysSpark dataframe for Reddit Data
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql.functions import col, asc

# You can configure the SparkContext

conf = SparkConf()
conf.set('spark.local.dir', '/INET/state-trolls/work/state-trolls/poc/')
conf.set('spark.sql.shuffle.partitions', '2100')
SparkContext.setSystemProperty('spark.executor.memory', '10g')
SparkContext.setSystemProperty('spark.driver.memory', '10g')
sc = SparkContext(appName='mm_exp', conf=conf)
sqlContext = pyspark.SQLContext(sc)

In [95]:
reddit_data = sqlContext.read.json('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/RC_2016-03.bz2.decompressed')

In [15]:
import math
import glob
import json
range_counts = np.zeros(10)
l = np.empty((10, 0)).tolist()
f = []

In [34]:
i = 0
for name in glob.glob('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/annotations/formatted/RC_2016-03*'):
    print(i)
    with open(name, 'r') as read_file:
        jdata = json.load(read_file)
        df = pd.DataFrame(jdata)
        sample_df = df.sample(n = 17000)
        post_id = df.iloc[0]['post_id']
        post = reddit_data.where((col("id") == post_id)).collect()[0]
        for index, row in sample_df.iterrows():
            cos_sim = row['cosine_similarity']
            cos_sim = float(cos_sim)
            tweet_id = row['tweet_id']
            tweet = twitter_data.loc[twitter_data['tweetid'] == int(tweet_id)]
            tweet_text = tweet.iloc[0]['processed_tweets']
            range_index = 0
            if (cos_sim > 0):
                range_index = math.floor(cos_sim*10)
            if (cos_sim > 1):
                range_index = 9
            if (range_counts[range_index] < 50):
                range_counts[range_index] = range_counts[range_index] + 1
                d = {'tweet': tweet_text, 'post': post['body'], 'sim': cos_sim, 's_id': row['s_id']}
                l[range_index].append(d)
    i = i + 1
    print(range_counts)              

0
[ 50.  50.  50.  50.  50.  50.  24.   1.   0. 506.]
1


KeyboardInterrupt: 

In [ ]:
f.append(l)

In [107]:
range_counts

array([1500., 1500., 1500., 1500., 1500., 1500., 2500., 2500.,  948.,
       1480.])

In [106]:
i = 0
for name in glob.glob('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/annotations/formatted/RC_2016-03*'):
    if(range_counts[5] >1500 ):
        break
    print(i)
    with open(name, 'r') as read_file:
        jdata = json.load(read_file)
        df = pd.DataFrame(jdata)
        df = df.astype({'cosine_similarity':float})
        selected_df = df.loc[(df['cosine_similarity'] > 0.5) & (df['cosine_similarity'] <= 0.6)]
        print('len: ', len(selected_df))
        if(len(selected_df) > 1000):
            selected_df = selected_df.sample(n = 1000)
        if (len(selected_df)>0):
            post_id = df.iloc[0]['post_id']
            print(post_id)
            post = reddit_data.where((col("id") == post_id)).collect()[0]
            for index, row in selected_df.iterrows():
                cos_sim = row['cosine_similarity']
                cos_sim = float(cos_sim)
                tweet_id = row['tweet_id']
                tweet = twitter_data.loc[twitter_data['tweetid'] == int(tweet_id)]
                tweet_text = tweet.iloc[0]['processed_tweets']
                range_index = 0
                if (cos_sim > 0):
                    range_index = math.floor(cos_sim*10)
                if (cos_sim > 1):
                    range_index = 9
                if (range_counts[range_index] < 1500):
                    range_counts[range_index] = range_counts[range_index] + 1
                    d = {'tweet': tweet_text, 'post': post['body'], 'sim': cos_sim, 's_id': row['s_id']}
                    l[range_index].append(d)
        i = i + 1
           

0
len:  1697
d0ipu6y
1
len:  424
d0ipu97


KeyboardInterrupt: 

In [108]:
import random

f= []
for e in l:
    r = random.sample(e, 50)
    for i in r:
        f.append(i)

In [109]:
len(f)

500

In [110]:
random.shuffle(f)
f_df = pd.DataFrame(f)
# f_df.head()

s = []
for index, row in f_df.iterrows():
    post_text = row['post']
    sentences = nltk.sent_tokenize(post_text)  
    sentence = sentences[row['s_id']]
    s.append(sentence)
print(len(s))
f_df = f_df.assign(post_sentence = s)
f_df.head()

500


,tweet,post,sim,s_id,post_sentence
0,ItsTheEndOfTheWorldAnd I cant be bothered bec...,My point is that teams are making moves to fil...,0.498970,2,If your only reason for being good is that not...
1,Thank you,Thank you :),0.998981,0,Thank you :)
2,Thanks for holding,Thanks. I'll be sure to post similar things to...,0.711844,0,Thanks.
3,This is so unfair,I talked to a friend who owns a couple Chinese...,0.680722,9,And she ain't wrong.
4,thank you,Thanks. I'll be sure to post similar things to...,0.912710,0,Thanks.


In [111]:
f_df.to_csv('stsb_500_random_hits_v2.csv')

# Matrix for threshold analysis

In [116]:
m_df = pd.read_csv('/INET/state-trolls/work/state-trolls/poc/stsb-roberta-large/stsb_500_random_hits_v2.csv')
m_df2 = pd.read_csv('/INET/state-trolls/work/state-trolls/poc/stsb-roberta-large/stsb_copy2.csv') 

In [117]:
m_df.head()

,Unnamed: 0,tweet,post,sim,s_id,post_sentence
0,0,ItsTheEndOfTheWorldAnd I cant be bothered bec...,My point is that teams are making moves to fil...,0.498970,2,If your only reason for being good is that not...
1,1,Thank you,Thank you :),0.998981,0,Thank you :)
2,2,Thanks for holding,Thanks. I'll be sure to post similar things to...,0.711844,0,Thanks.
3,3,This is so unfair,I talked to a friend who owns a couple Chinese...,0.680722,9,And she ain't wrong.
4,4,thank you,Thanks. I'll be sure to post similar things to...,0.912710,0,Thanks.


In [120]:
annotation_df = pd.DataFrame()
annotation_df['annotation-1'] = m_df2['annotation']
annotation_df['cos-sim'] = m_df['sim']

In [132]:
annotation_df.head()

,annotation-1,cos-sim,true-values
0,0,0.498970,0
1,1,0.998981,1
2,1,0.711844,1
3,0,0.680722,1
4,1,0.912710,1


In [147]:
matrix_0 = []
matrix_1 = []
from sklearn.metrics import classification_report
target_names = ['irrelevant', 'relevant']
for threshold in np.arange(0.05, 1.0, 0.05):
    annotation_df['true-values'] = np.where(annotation_df['cos-sim'] > threshold, 1, 0)
    c = classification_report(annotation_df['true-values'],
                              annotation_df['annotation-1'], target_names=target_names, output_dict = True)
    matrix_0.append({'threshold': threshold, 'class': '0',
                   'precision' : (c['irrelevant'])['precision'],
                  'recall': (c['irrelevant'])['recall'],
                  'f1-score': (c['irrelevant'])['f1-score'],
                  'support': (c['irrelevant'])['support'],
                  'accuracy': c['accuracy']})
    matrix_1.append({'threshold': threshold, 'class': '1',
                   'precision' : (c['relevant'])['precision'],
                  'recall': (c['relevant'])['recall'],
                  'f1-score': (c['relevant'])['f1-score'],
                  'support': (c['relevant'])['support'],
                  'accuracy': c['accuracy']})
#     print(c)

In [136]:
matrix

[{'threshold': 0.05,
  'stats': {'irrelevant': {'precision': 0.03353658536585366,
    'recall': 0.9166666666666666,
    'f1-score': 0.06470588235294118,
    'support': 12},
   'relevant': {'precision': 0.9941860465116279,
    'recall': 0.35040983606557374,
    'f1-score': 0.5181818181818182,
    'support': 488},
   'accuracy': 0.364,
   'macro avg': {'precision': 0.5138613159387407,
    'recall': 0.6335382513661202,
    'f1-score': 0.2914438502673797,
    'support': 500},
   'weighted avg': {'precision': 0.9711304594441293,
    'recall': 0.364,
    'f1-score': 0.5072983957219251,
    'support': 500}}},
 {'threshold': 0.1,
  'stats': {'irrelevant': {'precision': 0.14939024390243902,
    'recall': 0.98,
    'f1-score': 0.25925925925925924,
    'support': 50},
   'relevant': {'precision': 0.9941860465116279,
    'recall': 0.38,
    'f1-score': 0.54983922829582,
    'support': 450},
   'accuracy': 0.44,
   'macro avg': {'precision': 0.5717881452070335,
    'recall': 0.6799999999999999,
   

In [149]:
pd.DataFrame(matrix_1).to_csv('threshold-analysis-matrix.csv')